Temporary dump location to resolve the issue of me forgetting to push some code and writing across two devices

To do:
 - ODEint warning is from a Stiff Equation. "hole" or jump in our solution points that occurs from having too wide timesteps
May want to look into other integration methods to resolve stiffness https://docs.scipy.org/doc/scipy-1.7.1/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp
Radau and BDF are recomended

Moving to irregular grid
For each initial condition (and from REAL SOLUTION)
 - arbitrary # of time points (big enough)
 - arbitrary spacing between them
 - sample section @ those time points
 - fit a polynomial (cubic spline)
 - sample from the fit on a regular grid
 - Run SINDy on regular grid fit
 - test the solution MSE (SINDy - real)
 - MSE (polynomial values - real) as an extra
 
 

In [ ]:
# Dynamic System #1 DMiB page 120
# Gene Network switch
u0 = 1
v0 = 1
T = 50
dt = 0.01
trange = np.array([0, T])
#tvals = np.arange(0, T + dt, dt)
tvals = np.linspace(0, T, 500)
fig, axs = plt.subplots(figsize=(10,8))
netswitch_x0s = (np.random.rand(n_of_ic, 2) * 5 - np.random.rand(n_of_ic, 2))
netswitch_train_multi = []
n_of_ic = 1000

# creation of data/solution
for i in range(n_of_ic): 
    #x0 = np.array([u_ic[i], v_ic[i]])
    # solve Sys #1
    #soln = solve_ivp(netswitch, trange, x0, t_eval=tvals)
    #uSoln = soln.y[0,:].reshape(-1,1)
    #vSoln = soln.y[1,:].reshape(-1,1)
    # graph 
    #axs.plot(tvals, uSoln, "r", label="$u$", **plot_kws)
    #axs.plot(tvals, vSoln, "b", label="$v$", **plot_kws)
    
    # phase portrait
    #axs.plot(uSoln, vSoln, label=f"$u_0 = {round(v_ic[i], 3)}, v_0 = {round(v_ic[i], 3)}$", alpha=0.4)
    
    # pySINDy
    # data creation
    netswitch_train_temp = solve_ivp(
        netswitch, trange, netswitch_x0s[i], t_eval=tvals
    ).y.T
    noise = np.random.normal(0, sigma, netswitch_train_temp.shape)
    netswitch_train_multi.append(netswitch_train_temp + noise)
    #netswitch_train_multi.append(netswitch_train_temp + np.random.normal(0, 0.015, netswitch_train_temp.shape))
#axs.legend()
#axs.set(xlabel="u", ylabel="$v$")
#plt.show()
#print(netswitch_train_multi)
test = netswitch_train_multi[0][:, 0]
print(netswitch_train_multi[0])
"""
sparse_regression_optimizer = ps.STLSQ(threshold=0.7)
model = ps.SINDy(feature_names=['u', 'v'], optimizer=sparse_regression_optimizer, feature_library=netswitch_library)
model.fit(netswitch_train_multi, t=dt, multiple_trajectories=True)
model.print()
plot_data_and_derivative(netswitch_train_multi[0], dt, ps.SmoothedFiniteDifference()._differentiate)
netswitch_sim = model.simulate(np.random.rand(2) * 5 - np.random.rand(2), t=tvals)
print(netswitch_sim.shape)
"""
#axs.plot(tvals, netswitch_sim[, "b", label="$v$", **plot_kws)
#netswitch_train_multi = []
#MSE_result = []
#for j in range(len(sigma_list)):
#    for i in range(n_of_ic): 
#        netswitch_train_temp = solve_ivp(
#            netswitch, trange, netswitch_x0s[i], t_eval=tvals
#        ).y.T
#        noise = np.random.normal(0, sigma_list[j], netswitch_train_temp.shape)
#        netswitch_train_multi.append(netswitch_train_temp + noise)
#    sparse_regression_optimizer = ps.STLSQ(threshold=0.7)
#    model = ps.SINDy(feature_names=['u', 'v'], optimizer=sparse_regression_optimizer, feature_library=netswitch_library)
#    model.fit(netswitch_train_multi, t=dt, multiple_trajectories=True)
#    netswitch_sim = model.simulate(np.random.rand(2) * 5 - np.random.rand(2), t=tvals)
    

In [ ]:
# code snippet, run w/ len(tvals) 500 and 5000 to see difference
# will need supporting code tho
test
htest = 1 / (1+test**5)
#plt.scatter(test, htest)
feature_count =2
netswitch_x0s = (np.random.rand(n_of_ic, feature_count) * 5 - np.random.rand(n_of_ic, feature_count))
model = ps.SINDy(feature_names=['u', 'v'], optimizer=ps.STLSQ(threshold=0.7), feature_library=netswitch_library)
model.fit(list(netswitch_train_multi), t=dt, multiple_trajectories=True)
model.print()
netswitch_sim = model.simulate(netswitch_x0s[0], t=tvals, integrator="odeint")
test2 = netswitch_sim[:, 0]
htest2 = 1 / (1+test2**5)
plt.scatter(tvals, test2)